# Compare Time Domain Survey and VASCA catalogs

### Setup main notebook options

In [1]:
region_name = "ALL_10-800_LOOSE" #"TDS_ELAISN1"  #"ALL_10-800_LOOSE" #
redo_src_sel = False  # Redo source selection in dedicated cell below

### Load VASCA region

In [2]:
# General path setup and library loading
%matplotlib widget
import matplotlib.pyplot as plt
from scipy.stats import chi2
import numpy as np

from astropy.coordinates import SkyCoord, match_coordinates_sky
from astropy import units as uu
from astropy.table import Table
from astropy.io import fits

from loguru import logger

from vasca.region import Region
from vasca.field import BaseField
from vasca.utils import sel_sources
import vasca.visualization as vvis

#Load region
region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+"_cat.fits"
rg = Region()
rg.load_from_fits(region_fname)
display(rg.tt_fields[:2])

#Apply source selections
if redo_src_sel:
    sel_vasca =  sel_sources(rg.tt_sources)
    rg.tt_sources["sel"][:]=sel_vasca   
else:
    sel_vasca = rg.tt_sources["sel"]
tt_vasca = rg.tt_sources[sel_vasca]

field_id,field_name,project,ra,dec,observatory,obs_filter,fov_diam,sel,nr_vis,time_bin_size_sum,time_start,time_stop,rg_fd_id
,,,deg,deg,,,deg,,,s,d,d,
bytes32,bytes32,bytes32,float64,float64,bytes22,bytes8,float32,bool,int32,float32,float64,float64,int64
GNU2509140647668088832,PKS2155m304,DIS,329.7262466081879,-30.2657589090371,GALEX,NUV,1.2,True,29,32261.951,54683.55502314815,55140.15796296296,1
GNU2509211016412266496,HPQ_3C454p3,DIS,343.48735799747396,16.1446550539213,GALEX,NUV,1.2,True,12,10216.749,54738.00383101852,56251.395905309604,2


### Get TDS & VASCA data and match them positionally

In [3]:
print(f"Selected {len(tt_vasca)} out of {len(rg.tt_sources)} sources")

#Read TDS
tt_tds_all = Table.read("./resources/TDS-ApJ_766_60/table4.fit", hdu="J_ApJ_766_60_table4")
#display(tt_tds_all)
ids = tt_tds_all["ID"].data.astype(dtype=str)
sel_elai = (np.char.find(ids, 'ELAI') > -1)
tt_tds = tt_tds_all
if "ELAISN1" in region_fname:    # restrict t ELAISN1 fields?
    tt_tds = tt_tds_all[sel_elai]
tt_tds['RAJ2000'].name = 'ra'
tt_tds['DEJ2000'].name = 'dec'

#Match VASCA-TDS
pos_tds = SkyCoord(ra=tt_tds["ra"], dec=tt_tds["dec"], unit="deg", frame="icrs")
pos_vasca = SkyCoord(ra=tt_vasca["ra"], dec=tt_vasca["dec"], unit="deg", frame="icrs")

match_dist = 10 * uu.arcsec
idx_tds, dist_tds, _ = pos_vasca.match_to_catalog_sky(pos_tds)
tt_vasca["tds_recno"]=tt_tds[idx_tds]["recno"]
tt_vasca["tds_idx"]=idx_tds
tt_vasca["tds_dist"]=dist_tds.to("arcsec")
tt_vasca["tds_match"] = tt_vasca["tds_dist"] < match_dist

idx_vasca, dist_vasca, _ = pos_tds.match_to_catalog_sky(pos_vasca)
tt_tds["rg_src_id"]=tt_vasca[idx_vasca]["rg_src_id"]
tt_tds["vasca_idx"]=idx_vasca
tt_tds["vasca_dist"]=dist_vasca.to("arcsec")
tt_tds["vasca_match"] = tt_tds["vasca_dist"] < match_dist

#display("Selected rg_src_ids",tt_vasca[:]["rg_src_id"].data)
#display(tt_tds[:2])

sel_tmv = tt_tds["vasca_match"]
sel_vmt = tt_vasca["tds_match"]
print("Nr. sources VASCA/TDS :",len(tt_vasca),"/",len(tt_tds))
print("Nr. associated sources VASCA/TDS:",sel_vmt.sum(),"/",sel_tmv.sum())
print("Fraction of associated VASCA/TDS sources:",sel_vmt.sum()/len(tt_vasca),"/",sel_tmv.sum()/len(tt_tds))

Selected 10413 out of 10413 sources
Nr. sources VASCA/TDS : 10413 / 1078
Nr. associated sources VASCA/TDS: 679 / 683
Fraction of associated VASCA/TDS sources: 0.06520695284740229 / 0.6335807050092764


In [4]:
#PS_ELAISN1_MOS15 is field_id GNU2597664528044916736 (TDE PS1-10jh field)
field_id = "GNU2597664528044916736" #rg.tt_fields["field_id"][0]
fd = rg.get_field(field_id=field_id, load_method="FITS", add_field=False)

fig, ax = vvis.plot_field_sky_map(fd, img_idx=-1, sky_region = fd.get_sky_region()) # 
ax = vvis.plot_sky_sources(rg.tt_sources,tt_det=rg.tt_detections, src_id="rg_src_id", sky_region_wcs = (fd.get_sky_region(),fd.ref_wcs), draw_labels=False)

plot_tds = {"marker":"x","color":"k"}
ax = vvis.plot_sky_sources(tt_tds,  src_id="recno",src_kwargs=plot_tds, sky_region_wcs = (fd.get_sky_region(),fd.ref_wcs), draw_labels=False) #

FileNotFoundError: [Errno 2] No such file or directory: './vasca_pipeline/ALL_10-800_LOOSE/fields/field_GNU2597664528044916736.fits'